In [2]:
import requests
import os

API_KEY = os.getenv("API_KEY")

SPORT = 'soccer_epl' 
REGIONS = 'uk' 
MARKETS = 'h2h' 
ODDS_FORMAT = 'decimal' 
DATE_FORMAT = 'iso' 
BETTING_EXCHANGES = set(["Matchbook", "Smarkets", "Betfair"])

SOCCER_KEYS = [
    "soccer_epl","soccer_efl_champ","soccer_england_league1","soccer_england_league2","soccer_england_efl_cup",
    "soccer_spl","soccer_league_of_ireland",
    "soccer_spain_la_liga","soccer_spain_segunda_division",
    "soccer_france_ligue_one","soccer_france_ligue_two",
    "soccer_germany_bundesliga","soccer_germany_bundesliga2","soccer_germany_liga3",
    "soccer_italy_serie_a","soccer_italy_serie_b",
    "soccer_portugal_primeira_liga","soccer_netherlands_eredivisie","soccer_belgium_first_div",
    "soccer_turkey_super_league","soccer_greece_super_league","soccer_switzerland_superleague",
    "soccer_austria_bundesliga","soccer_denmark_superliga","soccer_norway_eliteserien",
    "soccer_sweden_allsvenskan","soccer_sweden_superettan","soccer_finland_veikkausliiga","soccer_poland_ekstraklasa",
    "soccer_argentina_primera_division","soccer_brazil_campeonato","soccer_brazil_serie_b",
    "soccer_chile_campeonato","soccer_china_superleague","soccer_japan_j_league","soccer_korea_kleague1",
    "soccer_mexico_ligamx","soccer_usa_mls",
    "soccer_uefa_champs_league","soccer_uefa_europa_league",
    "soccer_conmebol_copa_libertadores","soccer_conmebol_copa_sudamericana",
    "soccer_australia_aleague",
]


In [3]:
odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()


In [4]:
import pandas as pd
from collections import defaultdict

leagues = []
for SPORT in SOCCER_KEYS:
    odds_response = requests.get(
        f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
        params={
            'api_key': API_KEY,
            'regions': REGIONS,
            'markets': MARKETS,
            'oddsFormat': ODDS_FORMAT,
            'dateFormat': DATE_FORMAT,
            'sport': SPORT,
            "commenceTimeTo": "2025-12-31T23:59:59Z"

        }
    )
    
    if odds_response.status_code != 200:
        print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

    else:
        odds_json = odds_response.json()

    df = pd.json_normalize(
        odds_json,
        record_path=['bookmakers','markets','outcomes'],
        meta=['id','sport_key','sport_title','commence_time','home_team','away_team',
                ['bookmakers','title'], ['bookmakers','last_update'],
                ['bookmakers','markets','last_update']],
        errors='ignore'
    )

    leagues.append(df)

parent = pd.concat(leagues)

print('Remaining requests', odds_response.headers['x-requests-remaining'])
print('Used requests', odds_response.headers['x-requests-used'])


Remaining requests 46
Used requests 454


In [5]:
parent = (parent.rename(columns={
        'id':'event_id',
        'bookmakers.title':'bookmaker',
        'bookmakers.last_update':'bookmaker_last_update',
        'bookmakers.markets.last_update':'market_last_update',
        'name':'outcome_name',
        'price':'odds_decimal'
    })
    [['event_id','commence_time','home_team','away_team',
        'bookmaker','bookmaker_last_update',
        'outcome_name','odds_decimal']]
)

parent.to_csv("current_data.csv")


In [6]:
#{(home_team, away_team): {outcome: [(odds, bookmaker), ...]}}
odds_by_match = {}

for _, row in parent.iterrows():
    match_key = (row['home_team'], row['away_team'])
    outcome = row['outcome_name']
    odds = row['odds_decimal']
    bookmaker = row['bookmaker']
    
    if match_key not in odds_by_match:
        odds_by_match[match_key] = {}
    
    if outcome not in odds_by_match[match_key]:
        odds_by_match[match_key][outcome] = []
    
    odds_by_match[match_key][outcome].append((odds, bookmaker))


## Back-to-back arbitrage

Back-to-back arbitrage means building a hedge by taking complementary positions across a betting exchange and one or more bookmakers so that, once bets are matched, the total exposure is hedged and a guaranteed profit (or lower risk) is achieved.

### How it's implemented
- Collect odds for each outcome from multiple bookmakers and exchanges.
- For each match with three mutually exclusive outcomes, enumerate candidate combinations picking one quote per outcome.
- Compute H = sum(1 / odds_i) for the chosen combination. If H < 1 (adjusted by `epsilon` to cover fees/slippage) an arbitrage exists.
- Slippage is the risk of the odds fluctuating before we can fill the order
- Flag combinations that include a betting exchange because there is higher slippage and less volume.
- Sort opportunities by whether they include exchanges and by `h_val`.

### Stake allocation (guarantees equal payout)
- Let bank = total amount committed.
- stake_i = bank * (1 / odds_i) / H
- payout = bank / H  (same regardless of which outcome wins)
- profit = payout - bank = bank * (1/H - 1)

### Practical caveats
- Exchanges have commission and different liquidity — we adjust `epsilon` and verify available matched amounts before committing full stakes.
- Bookmakers can limit or void bets.
- Latency and partial fills break hedges. E.g Exchange bet first.
- Currency, min/max stake and market rules differ.

In short: find 3-way combinations with H < 1 and compute stake splits to equalise payouts.

In [23]:
import pprint

BETTING_EXCHANGES = set(["Matchbook", "Smarkets", "Betfair"])

# all bookmakers in GDP, assuming 1000 max bankroll
bankroll = 1000

# estimate to account for slippage and fees
epsilon = 0.005

def calc_h_3_way(odds1, odds2, odds3):
    return (1/(odds1)) + (1/(odds2)) + (1/(odds3))

def calculate_stakes_for_combination(combination, bankroll):

    odds_list = [v[0] for v in combination.values()]

    # keeps this function general for reuse
    h = sum((1 / o) for o in odds_list)

    stakes = {outcome: bankroll * (1 / odds) / h for outcome, (odds, _) in combination.items()}

    payout = bankroll / h
    profit = payout - bankroll

    return {
        'stakes': stakes,
        'payout': payout,
        'profit': profit,
    }

def find_arbritrage(odds_by_match):
    oppourtunities = []
    for match_key, outcomes in odds_by_match.items():

        if len(outcomes) != 3:
            continue

        best_h_val = 1 - epsilon
        current_best = None

        best_h_val_with_exchanges = 1 - epsilon
        current_best_with_exchanges = None
    
        outcome1, outcome2, outcome3 = outcomes.keys()

        # brute force all combinations because small search space
        for odds1, bookie1 in outcomes[outcome1]:
            for odds2, bookie2 in outcomes[outcome2]:
                for odds3, bookie3 in outcomes[outcome3]:

                    if len({bookie1, bookie2, bookie3}) >= 2:
                        h_val = calc_h_3_way(odds1, odds2, odds3)
                        if h_val < best_h_val and not bool({bookie1, bookie2, bookie3} & BETTING_EXCHANGES):
                            best_h_val = h_val
                            current_best = {
                                outcome1: (odds1, bookie1),
                                outcome2: (odds2, bookie2),
                                outcome3: (odds3, bookie3)
                            }
                        elif h_val < best_h_val:
                            best_h_val_with_exchanges = h_val
                            current_best_with_exchanges = {
                                outcome1: (odds1, bookie1),
                                outcome2: (odds2, bookie2),
                                outcome3: (odds3, bookie3)
                            }

        if current_best:
            # funcfact as of 
            stakes, payout, profit = calculate_stakes_for_combination(current_best, bankroll).values()
            profit_margin = ((1/best_h_val)-1)*100
            contains_betting_exchange = False
            oppourtunities.append({
                'match': f"{match_key[0]} vs {match_key[1]}",
                'h_val': best_h_val,
                'combination': current_best,
                'stakes': stakes,
                'profit_margin_%': profit_margin,
                'contains_betting_exchange': contains_betting_exchange,
                'payout': payout,
                'profit': profit,
            })

        if current_best_with_exchanges:
            stakes, payout, profit = calculate_stakes_for_combination(current_best_with_exchanges, bankroll).values()
            profit_margin = ((1/best_h_val_with_exchanges)-1)*100
            contains_betting_exchange = True
            oppourtunities.append({
                'match': f"{match_key[0]} vs {match_key[1]}",
                'h_val': best_h_val_with_exchanges,
                'combination': current_best_with_exchanges,
                'stakes': stakes,
                'profit_margin_%': profit_margin,
                'contains_betting_exchange': contains_betting_exchange,
                'payout': payout,
                'profit': profit,
            })
    
    return sorted(oppourtunities, key=lambda x: (x['contains_betting_exchange'], x['h_val']))
    
arb_opps = find_arbritrage(odds_by_match)
print(f"Assuming {bankroll} bankroll:\n")

for item in arb_opps:
    pprint.pprint(item, sort_dicts=False)
    print()


Assuming 1000 bankroll:

{'match': 'Sturm Graz vs Wolfsberger AC',
 'h_val': 0.8048245614035088,
 'combination': {'Sturm Graz': (4.8, 'Unibet (UK)'),
                 'Wolfsberger AC': (3.0, 'William Hill'),
                 'Draw': (3.8, 'Unibet (UK)')},
 'stakes': {'Sturm Graz': 258.8555858310627,
            'Wolfsberger AC': 414.16893732970027,
            'Draw': 326.97547683923705},
 'profit_margin_%': 24.250681198910073,
 'contains_betting_exchange': False,
 'payout': 1242.506811989101,
 'profit': 242.50681198910092}

{'match': 'Sturm Graz vs Wolfsberger AC',
 'h_val': 0.7195099824189299,
 'combination': {'Sturm Graz': (5.9, 'Matchbook'),
                 'Wolfsberger AC': (3.15, 'Betfair'),
                 'Draw': (4.3, 'Matchbook')},
 'stakes': {'Sturm Graz': 235.56521739130434,
            'Wolfsberger AC': 441.21739130434787,
            'Draw': 323.2173913043478},
 'profit_margin_%': 38.98347826086959,
 'contains_betting_exchange': True,
 'payout': 1389.8347826086958,
 'pr